In [19]:
import pickle  
import json

import xml.etree.ElementTree as etree
import spacy
import json
# from gensim.models import Word2Vec
import numpy as np
# import pandas as pd
# from pandas import Series
import logging
import numpy
from nltk.corpus import wordnet as wn
import os

import csv


import en_core_web_sm

Utilize pairs count data:

In [20]:
f = open('corpus_stats.pkl','rb')  
corpus_stats = pickle.load(f)  
print (type(corpus_stats) )  #show file 

<class 'dict'>


In [21]:
corpus_stats.keys()

dict_keys(['word2id', 'id2word', 'word_freq', 'vocab_size', 'num_pairs', 'num_sentences'])

In [22]:
corpus_stats["vocab_size"]

100026

In [23]:
f = open('pairs_count.pkl','rb')  
pairs_count = pickle.load(f)  
print (type(pairs_count) )  #show file 

<class 'dict'>


In [24]:
pairs_count.keys()

dict_keys(['amod', 'nsubj', 'dobj', 'nsubj_amod', 'dobj_amod'])

In [25]:
# pairs_count["nsubj_amod"]

In [26]:
print(pairs_count["amod"][(17, 19)])

14917


In [27]:
import numpy as np

bar_dict = dict()

for key in pairs_count.keys():

    data = np.array(list(pairs_count[key].values()))
    # print(np.mean(data))
    s = np.sort(data)

    bar_dict[key] = s[-100000]
    
print(bar_dict)

{'amod': 139, 'nsubj': 77, 'dobj': 77, 'nsubj_amod': 17, 'dobj_amod': 23}


Utilize knowledge graph data:

In [28]:
# # !pip install -r requirements.txt
# # !python setup.py develop
# from aser.database.db_API import KG_Connection
# from aser.database.db_API import generate_event_id, generate_relation_id, generate_id

In [29]:
# kg_conn = KG_Connection(db_path=r'KG_wino.db', mode='cache')
# print('SIZE:', 'eventualities: ', len(kg_conn.event_id_set), 'relations:', len(kg_conn.relation_id_set))

In [30]:
# event_id_list = list(kg_conn.event_id_set)
# event_id_list[0]

In [31]:
# kg_conn.get_exact_match_event(event_id_list[0])

In [32]:
# print(list(zip(kg_conn.event_columns, kg_conn.event_column_types)))

Utilize SP-10K data

In [33]:
path = '.\sp10k_data'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)
    
def sp_tsv2wino(filename):
    with open(filename) as tsvfile:
        
        data_list = []
        
        reader = csv.reader(tsvfile, dialect='excel-tab')
        
        if "nsubj" in filename:
            if "amod" in filename:
                data_type = "s-a"
            else:
                data_type = "s"
        elif "dobj" in filename:
            if "amod" in filename:
                data_type = "o-a"
            else:
                data_type = "o"
        else:
            data_type = "a"
        
        print(data_type)
        
        for row in reader:
            if data_type == "s-a":
                temp = []
                text = "A "+ row[0] + " B, " + "he be " + row[1]
                temp.append(text)
                temp.append("A")
                data_list.append(temp)
            elif data_type == "o-a":
                temp = []
                text = "A "+ row[0] + " B, " + "he be " + row[1]
                temp.append(text)
                temp.append("B")
                data_list.append(temp)
        
        return data_list

def list2txt(data_list, filename):
    file = open(filename,"w")
    
    for sample in data_list:
        file.write(sample[0])
        file.write("\n")
        file.write("he")
        file.write("\n")
        file.write("A")
        file.write(",")
        file.write("B")
        file.write("\n")
        file.write(sample[1])
        file.write("\n")
        file.write("\n")
    
# data_list = sp_tsv2wino(files[1])
# print(len(data_list))
# list2txt(data_list, "test.txt")

.\sp10k_data\amod_annotation.txt
.\sp10k_data\dobj_amod_annotation.txt
.\sp10k_data\dobj_annotation.txt
.\sp10k_data\nsubj_amod_annotation.txt
.\sp10k_data\nsubj_annotation.txt
.\sp10k_data\wino_dobj_amod_annotation.txt
.\sp10k_data\wino_nsubj_amod_annotation.txt


Take the wino data:

In [34]:
# !python -m spacy download en_core_web_sm

In [35]:
# Load WSC dataset


tree = etree.parse('WSCollection.xml')
root = tree.getroot()
original_problems = root.getchildren()
problems = list()

for original_problem in original_problems:
    problem = dict()
    for information in original_problem.getchildren():
        
        if information.tag == 'answers':
            answers = information.getchildren()
            answer_list = list()
            for answer in answers:
                answer_list.append(answer.text.strip())
            problem['answers'] = answer_list
        elif information.tag == 'text':
            texts = information.getchildren()
            # print(texts)
            text_dict = dict()
            for text1 in texts:
                text_dict[text1.tag] = text1.text.replace('\n', ' ').strip()
            problem['text'] = text_dict
        elif information.tag == 'quote':
            pass
        else:
            problem[information.tag] = information.text.replace(' ', '')
    problems.append(problem)

# all_sentences = list()
# for question in problems:
#     sentence = question['text']['txt1'] + ' ' + question['text']['pron'] + ' ' + question['text']['txt2']
#     all_sentences.append(sentence)

print(problems[0])

# from nltk.stem.snowball import SnowballStemmer
# stemmer = SnowballStemmer("english")


nlp = en_core_web_sm.load()

# stemmer.stem("running")

for question in problems:
    sentence = question['text']['txt1'] + ' ' + question['text']['pron'] + ' ' + question['text']['txt2']
    doc = nlp(sentence)
    verbs = [token.lemma_ for token in doc if token.pos_ == "VERB"]
    # stem_verbs = []
    cand1_doc = nlp(question["answers"][0])
    cand2_doc = nlp(question["answers"][1])
    
    noun1= [token.lemma_ for token in cand1_doc if (token.pos_ in ["NOUN","PROPN"])]
    noun2= [token.lemma_ for token in cand2_doc if (token.pos_ in ["NOUN","PROPN"])]
    
    if not noun1:
        noun1 = [question["answers"][0]]
    if not noun2:
        noun2 = [question["answers"][1]]
        
    question["verbs"] = verbs
    question["noun1"] = noun1
    question["noun2"] = noun2
    
print(problems[0])
    

C:\Users\xzhao\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  
C:\Users\xzhao\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\xzhao\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
C:\Users\xzhao\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  


{'text': {'txt1': 'The city councilmen refused the demonstrators a permit because', 'pron': 'they', 'txt2': 'feared violence.'}, 'answers': ['The city councilmen', 'The demonstrators'], 'correctAnswer': 'A', 'source': '(Winograd1972)'}
{'text': {'txt1': 'The city councilmen refused the demonstrators a permit because', 'pron': 'they', 'txt2': 'feared violence.'}, 'answers': ['The city councilmen', 'The demonstrators'], 'correctAnswer': 'A', 'source': '(Winograd1972)', 'verbs': ['refuse', 'fear'], 'noun1': ['city', 'councilman'], 'noun2': ['demonstrator']}


In [36]:
# sample of the stored data
sample = dict()
sample["text"] = ""
sample["candA"] = ""
sample["candB"] = ""
sample["ans"] = "A"
sample["pron"] = "he"
sample["source"] = "" # ["aser","sp10k","pairs_count"]
sample["format"] = "" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
sample["score"] = 0 # importance, not yet used
sample["coverage"] = 0 # portion of the tokens in the orignal question coverged

In [37]:
# initialize a data collection for all the data
data_collection = dict()
for i in range(273):
    data_collection[i] = []
    
# construct a related words sheet for all 273 queries
related_words = dict()
related_words["noun1"] = []
related_words["noun2"] = []
related_words["verbs"] = []

for i in range(273):
    question = problems[i]
    related_words["noun1"].append(question["noun1"])
    related_words["noun2"].append(question["noun2"])
    related_words["verbs"].append(question["verbs"])


In [38]:
# let's start from sp10k

path = '.\sp10k_data'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)
    
sp_data_collection = dict()
for i in range(273):
    sp_data_collection[i] = []
    
def sp_util(filename, data_collection, related_words, problems):
    with open(filename) as tsvfile:
        
        reader = csv.reader(tsvfile, dialect='excel-tab')
        
        if "nsubj" in filename:
            if "amod" in filename:
                data_type = "s-a"
            else:
                data_type = "s"
        elif "dobj" in filename:
            if "amod" in filename:
                data_type = "o-a"
            else:
                data_type = "o"
        else:
            data_type = "a"
        
        print(data_type)
        
        for row in reader:
            if data_type == "s-a":
                for i in range(273):
                    noun1 = related_words["noun1"][i]
                    noun2 = related_words["noun2"][i]
                    verbs = related_words["verbs"][i]
                    
                    candA = problems[i]["answers"][0]
                    candB = problems[i]["answers"][1]
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = candA + " " + row[0] + " "  + candB + ", he be " + row[1] + "."
                        sample["candA"] = candA
                        sample["candB"] = candB
                        sample["ans"] = "A"
                        sample["pron"] = "he"
                        sample["source"] = "sp10k" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "nsubj_amod" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage      
                        
                        data_collection[i].append(sample)
                        
            elif data_type == "o-a":
                for i in range(273):
                    noun1 = related_words["noun1"][i]
                    noun2 = related_words["noun2"][i]
                    verbs = related_words["verbs"][i]
                    
                    candA = problems[i]["answers"][0]
                    candB = problems[i]["answers"][1]
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = candA + " " + row[0] + " " + candB + ", he be " + row[1] + "."
                        sample["candA"] = candA
                        sample["candB"] = candB
                        sample["ans"] = "B"
                        sample["pron"] = "he"
                        sample["source"] = "sp10k" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "dobj_amod" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
            elif data_type == "s":
                for i in range(273):
                    noun1 = related_words["noun1"][i]
                    noun2 = related_words["noun2"][i]
                    verbs = related_words["verbs"][i]
                    
                    candA = problems[i]["answers"][0]
                    candB = problems[i]["answers"][1]
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = row[1] + " and " + candB + ", he " + row[0] + "."
                        sample["candA"] = row[1]
                        sample["candB"] = candB
                        sample["ans"] = "A"
                        sample["pron"] = "he"
                        sample["source"] = "sp10k" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "nsubj" # ["amod","v","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = row[1] + " and " + candA + ", he " + row[0] + "."
                        sample["candA"] = candA
                        sample["candB"] = row[1]
                        sample["ans"] = "B"
                        sample["pron"] = "he"
                        sample["source"] = "sp10k" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "nsubj" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
            elif data_type == "o":
                for i in range(273):
                    noun1 = related_words["noun1"][i]
                    noun2 = related_words["noun2"][i]
                    verbs = related_words["verbs"][i]
                    
                    candA = problems[i]["answers"][0]
                    candB = problems[i]["answers"][1]
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = row[1] + " and " + candB + ", " + row[0] + " he."
                        sample["candA"] = row[1]
                        sample["candB"] = candB
                        sample["ans"] = "A"
                        sample["pron"] = "he"
                        sample["source"] = "sp10k" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "dobj" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        data_collection[i].append(sample)
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = row[1] + " and " + candA + ", " + row[0] + " he."
                        sample["candA"] = candA
                        sample["candB"] = row[1]
                        sample["ans"] = "B"
                        sample["pron"] = "he"
                        sample["source"] = "sp10k" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "dobj" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
            elif data_type == "a":
                for i in range(273):
                    noun1 = related_words["noun1"][i]
                    noun2 = related_words["noun2"][i]
                    verbs = related_words["verbs"][i]
                    
                    candA = problems[i]["answers"][0]
                    candB = problems[i]["answers"][1]
                    
                    if row[0] in noun1:
                        sample = dict()
                        sample["text"] = candA + " and " + candB + ", he be " + row[1] + "."
                        sample["candA"] = candA
                        sample["candB"] = candB
                        sample["ans"] = "A"
                        sample["pron"] = "he"
                        sample["source"] = "sp10k" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "amod" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
                    if row[0] in noun2:
                        sample = dict()
                        sample["text"] = candA + " and " + candB + ", he be " + row[1] + "."
                        sample["candA"] = candA
                        sample["candB"] = candB
                        sample["ans"] = "B"
                        sample["pron"] = "he"
                        sample["source"] = "sp10k" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "amod" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
        
        return True

for f in files:
    if not "wino" in f:
        sp_util(f, sp_data_collection, related_words, problems)

.\sp10k_data\amod_annotation.txt
.\sp10k_data\dobj_amod_annotation.txt
.\sp10k_data\dobj_annotation.txt
.\sp10k_data\nsubj_amod_annotation.txt
.\sp10k_data\nsubj_annotation.txt
.\sp10k_data\wino_dobj_amod_annotation.txt
.\sp10k_data\wino_nsubj_amod_annotation.txt
a
o-a
o
s-a
s


In [55]:
# for i in range(273):
#     print(len(data_collection[i]))

# for data in data_collection[0]:
#     print(data["text"])

In [91]:
# then pairs count

def pc_util(data_collection, related_words, pairs_count,corpus_stats, bar_dict, problems):
    
    for raw_type in list(pairs_count.keys()):
        print(raw_type)
        if "nsubj" in raw_type:
            if "amod" in raw_type:
                data_type = "s-a"
            else:
                data_type = "s"
        elif "dobj" in raw_type:
            if "amod" in raw_type:
                data_type = "o-a"
            else:
                data_type = "o"
        else:
            data_type = "a"

        print(data_type)

        for key, value in pairs_count[raw_type].items():
            row = []
            row.append(corpus_stats["id2word"][key[0]])
            row.append(corpus_stats["id2word"][key[1]])
            row.append(str(value))
                
            if value < bar_dict[raw_type]:
                continue

            
            if data_type == "s-a":
                for i in range(273):
                    noun1 = related_words["noun1"][i]
                    noun2 = related_words["noun2"][i]
                    verbs = related_words["verbs"][i]
                    
                    candA = problems[i]["answers"][0]
                    candB = problems[i]["answers"][1]
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = candA + " " + row[0] + " "  + candB + ", he be " + row[1] + "."
                        sample["candA"] = candA
                        sample["candB"] = candB
                        sample["ans"] = "A"
                        sample["pron"] = "he"
                        sample["source"] = "pairs_count" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "nsubj_amod" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                        
            elif data_type == "o-a":
                for i in range(273):
                    noun1 = related_words["noun1"][i]
                    noun2 = related_words["noun2"][i]
                    verbs = related_words["verbs"][i]
                    
                    candA = problems[i]["answers"][0]
                    candB = problems[i]["answers"][1]
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = candA + " " + row[0] + " " + candB + ", he be " + row[1] + "."
                        sample["candA"] = candA
                        sample["candB"] = candB
                        sample["ans"] = "B"
                        sample["pron"] = "he"
                        sample["source"] = "pairs_count" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "dobj_amod" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
            elif data_type == "s":
                for i in range(273):
                    noun1 = related_words["noun1"][i]
                    noun2 = related_words["noun2"][i]
                    verbs = related_words["verbs"][i]
                    
                    candA = problems[i]["answers"][0]
                    candB = problems[i]["answers"][1]
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = row[1] + " and " + candB + ", he " + row[0] + "."
                        sample["candA"] = row[1]
                        sample["candB"] = candB
                        sample["ans"] = "A"
                        sample["pron"] = "he"
                        sample["source"] = "pairs_count" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "nsubj" # ["amod","v","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = row[1] + " and " + candA + ", he " + row[0] + "."
                        sample["candA"] = candA
                        sample["candB"] = row[1]
                        sample["ans"] = "B"
                        sample["pron"] = "he"
                        sample["source"] = "pairs_count"# ["aser","sp10k","pairs_count"]
                        sample["format"] = "nsubj" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
            elif data_type == "o":
                for i in range(273):
                    noun1 = related_words["noun1"][i]
                    noun2 = related_words["noun2"][i]
                    verbs = related_words["verbs"][i]
                    
                    candA = problems[i]["answers"][0]
                    candB = problems[i]["answers"][1]
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = row[1] + " and " + candB + ", " + row[0] + " he."
                        sample["candA"] = row[1]
                        sample["candB"] = candB
                        sample["ans"] = "A"
                        sample["pron"] = "he"
                        sample["source"] = "pairs_count" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "dobj" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
                    if row[0] in verbs:
                        sample = dict()
                        sample["text"] = row[1] + " and " + candA + ", " + row[0] + " he."
                        sample["candA"] = candA
                        sample["candB"] = row[1]
                        sample["ans"] = "B"
                        sample["pron"] = "he"
                        sample["source"] = "pairs_count" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "dobj" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
            elif data_type == "a":
                for i in range(273):
                    noun1 = related_words["noun1"][i]
                    noun2 = related_words["noun2"][i]
                    verbs = related_words["verbs"][i]
                    
                    candA = problems[i]["answers"][0]
                    candB = problems[i]["answers"][1]
                    
                    if row[0] in noun1:
                        sample = dict()
                        sample["text"] = candA + " and " + candB + ", he be " + row[1] + "."
                        sample["candA"] = candA
                        sample["candB"] = candB
                        sample["ans"] = "A"
                        sample["pron"] = "he"
                        sample["source"] = "pairs_count" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "amod" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)
                    
                    if row[0] in noun2:
                        sample = dict()
                        sample["text"] = candA + " and " + candB + ", he be " + row[1] + "."
                        sample["candA"] = candA
                        sample["candB"] = candB
                        sample["ans"] = "B"
                        sample["pron"] = "he"
                        sample["source"] = "pairs_count" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "amod" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(row[2]) # importance, not yet used
                        
                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 
                        
                        data_collection[i].append(sample)

    return True


pc_data_collection = dict()
for i in range(273):
    pc_data_collection[i] = []

pc_util(pc_data_collection, related_words, pairs_count,corpus_stats, bar_dict, problems)

amod
a
nsubj
s
dobj
o
nsubj_amod
s-a
dobj_amod
o-a


True

In [95]:
# for i in range(273):
#     print(len(pc_data_collection[i]))

# for data in pc_data_collection[0]:
#     print(data["text"])

In [15]:
path = '.\KG_by_question\\'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.json' in file:
            files.append(os.path.join(r, file))

# for f in files:
#     print(f)
    
with open(files[0]) as json_file:
    data = json.load(json_file)
    print(data[1])

{'_id': '5d5f2556c910aa10a390207f5f565d789af3563d', 'event1_id': '224308091a0c5def2f1292c84a366972e5feb666', 'event2_id': '214a330751eed3e386114ac068c96e316fc88dcc', 'Precedence': 0.0, 'Succession': 0.0, 'Synchronous': 0.0, 'Reason': 1.0, 'Result': 0.0, 'Condition': 0.0, 'Contrast': 0.0, 'Concession': 0.0, 'Conjunction': 0.0, 'Instantiation': 0.0, 'Restatement': 0.0, 'ChosenAlternative': 0.0, 'Alternative': 0.0, 'Exception': 0.0, 'Co_Occurrence': 1.0, 'event1': {'_id': '224308091a0c5def2f1292c84a366972e5feb666', 'verbs': 'place', 'skeleton_words_clean': 'place in water', 'skeleton_words': 'it place in water', 'words': 'it can not be place in the water', 'pattern': 'spass-v-X-o', 'frequency': 1.0}, 'event2': {'_id': '214a330751eed3e386114ac068c96e316fc88dcc', 'verbs': 'refuse', 'skeleton_words_clean': 'official refuse permission', 'skeleton_words': 'official refuse permission', 'words': 'united states port security official have refuse permission', 'pattern': 's-v-o', 'frequency': 1.0}}

In [ ]:
# finally from KG
lemma_stop_words = ["do","be"]

def kg_util(files, data_collection, related_words, problems, stop_words):
    
    for f in files:
        
        for i in range(273):
            if str(i+1) in f:
                rank = i
        # print(rank)
        
        
        try:
            with open(f) as json_file:
                data = json.load(json_file)
                a = 1
        except:
            continue
            
        
        with open(f) as json_file:
            
            data = json.load(json_file)
            
            for instance in data:
                
                if instance["Co_Occurrence"] < 10:
                    # print("my_faults")
                    continue
                
                sverbs = instance["event1"]["verbs"]
                sverbs += " "
                sverbs += instance["event2"]["verbs"]
                
                v_doc = nlp(sverbs)
                lemma_v = [token.lemma_ for token in v_doc]
                
                useless = 0
                for verb in lemma_v:
                    if verb in stop_words:
                        useless = 1
                        
                if useless:
                    # print("useless")
                    continue
                
                try:
                    event1_doc = nlp(instance["event1"]["skeleton_words"])
                    event2_doc = nlp(instance["event2"]["skeleton_words"])
                except:
                    continue
                
                obj1 = [token.lemma_ for token in event1_doc if (token.pos_ in ["PROPN", "NOUN"])]
                obj2 = [token.lemma_ for token in event2_doc if (token.pos_ in ["PROPN", "NOUN"])]
                    
                pairs = []
                unpaired = []
                
                for obj in obj1:
                    if obj in obj2:
                        pairs.append(obj)
                    else:
                        unpaired.append(obj)
                
                if not pairs:
                    # print("lol")
                    continue
                
                
                if len(unpaired) == 0:
                    for obj in obj2:
                        if obj not in pairs:
                            unpaired.append(obj)
                
                if len(unpaired) == 0:
                    unpaired.append("A")
                
                    
                for paired_obj in pairs:
                    try:
                        event1_text = instance["event1"]["skeleton_words"]
                        event2_text = instance["event2"]["skeleton_words"].replace(paired_obj, "[MASK]")

                        sample = dict()
                        sample["text"] = event1_text + ", " +  event2_text + "."                  

                        sample["candA"] = paired_obj
                        sample["candB"] = unpaired[0]
                        sample["ans"] = "A"
                        sample["pron"] = "[MASK]"
                        sample["source"] = "aser" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "aser" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(instance["Co_Occurrence"]) # importance, not yet used

                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage 

                        data_collection[rank].append(sample)
                    except:
                        continue
                    
                for paired_obj in pairs:
                    
                    try:
                        event1_text = instance["event1"]["skeleton_words"].replace(paired_obj, "[MASK]")
                        event2_text = instance["event2"]["skeleton_words"]

                        sample = dict()
                        sample["text"] = event2_text + ", " +  event1_text + "."                  

                        sample["candA"] = unpaired[0]
                        sample["candB"] = paired_obj
                        sample["ans"] = "B"
                        sample["pron"] = "[MASK]"
                        sample["source"] = "aser" # ["aser","sp10k","pairs_count"]
                        sample["format"] = "aser" # ["amod","nsubj","dobj","nsubj_amod","dobj_amod","aser"]
                        sample["score"] = str(instance["Co_Occurrence"]) # importance, not yet used

                        sentence = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
                        doc = nlp(sentence)
                        useful_tokens = [token.lemma_ for token in doc if token.pos_ in ["VERB","NOUN","PROPN","ADP","ADJ","ADV"]]
                        cov = 0
                        for token in useful_tokens:
                            if token in sample["text"]:
                                cov+=1
                        coverage = cov/len(useful_tokens)
                        sample["coverage"] = coverage      

                        data_collection[rank].append(sample)
                    except:
                        continue
                
kg_data_collection = dict()
for i in range(273):
    kg_data_collection[i] = []  
                
kg_util(files, kg_data_collection, related_words, problems, lemma_stop_words)              

In [ ]:
for i in range(273):
    print(len(kg_data_collection[i]))

for data in kg_data_collection[0]:
    print(data["text"])

In [ ]:
def rectifier(data_collection,format_list,source_list,score_dict):
    
    rec_data_collecton = dict()
    
    for i in range(273):
        rec_data_collecton[i] = []
        
        for sample in data_collection[i]:
            if not sample["source"] in source_list:
                continue
             
            if not sample["format"] in format_list:
                continue
                
            if float(sample["score"]) < float(score_dict[sample["source"]][sample["format"]]):
                continue
                
            rec_data_collecton[i].append(sample)
            
    return rec_data_collecton
            

In [53]:
def data2txt(data_collection, problems, filename):
    
    for i in range(273):
        
        path = "./"+filename+"/"+str(i)+"/"
        try:
            os.makedirs(path)
        except:
            a = 1
            
            
        file = open(path + "train.c.txt","w")

        for sample in data_collection[i]:
            file.write(sample["text"])
            file.write("\n")
            file.write(sample["pron"])
            file.write("\n")
            file.write(sample["candA"])
            file.write(",")
            file.write(sample["candB"])
            file.write("\n")
            
            if sample["ans"] == "A":
                label = sample["candA"]
            else:
                label = sample["candB"]
                
            file.write(label)
            file.write("\n")
            file.write("\n")
            
        file.close()
        
        file2 = open(path + "test.c.txt","w")
        
        text = problems[i]['text']['txt1'] + ' ' + problems[i]['text']['pron'] + ' ' + problems[i]['text']['txt2']
        
        file2.write(text)
        file2.write("\n")

        file2.write(problems[i]['text']['pron'])
        file2.write("\n")

        file2.write(problems[i]["answers"][0])
        file2.write(",")
        file2.write(problems[i]["answers"][1])
        file2.write("\n") 

        file2.write(problems[i]["correctAnswer"])
        file2.write("\n") 
        file2.write("\n") 
            
data2txt(sp_data_collection, problems, "sp")